In [1]:
import numpy as np
import pandas as pd
import os, sys, glob

In [2]:
floc = '/uscms/home/sbrightt/nobackup/iDM/analysis/CMSSW_10_2_18/src/iDMSkimmer/analysis/jose_signal/cutflow/'
csvs = glob.glob(floc+'*.csv')
names = [n.split('/')[-1].split('.')[0] for n in csvs]

In [3]:
names

['Mchi-22p0_dMchi-4p0_cutflow',
 'Mchi-44p0_dMchi-8p0_cutflow',
 'Mchi-5p5_dMchi-1p0_cutflow',
 'Mchi-80p4_dMchi-0p8_cutflow',
 'Mchi-100p5_dMchi-1p0_cutflow',
 'Mchi-10p5_dMchi-1p0_cutflow',
 'Mchi-110p0_dMchi-20p0_cutflow',
 'Mchi-11p0_dMchi-2p0_cutflow',
 'Mchi-33p0_dMchi-6p0_cutflow',
 'Mchi-40p2_dMchi-0p4_cutflow',
 'Mchi-60p3_dMchi-0p6_cutflow',
 'Mchi-66p0_dMchi-12p0_cutflow']

In [4]:
newtabs = {}
htmls = {}
for i, n in enumerate(csvs):
    name = names[i]
    tab = pd.read_csv(csvs[i],sep=" ")
    tab.drop(columns=['Data','Bkg'])
    cols = tab.columns
    pts = list(filter(lambda x: 'sig' in x,cols))
    new = {"Cut":tab["Description"]}
    for pt in pts:
        new[pt] = tab[pt]
        new[pt+'_eff'] = 100*tab[pt]/tab[pt][0]
    tab2 = pd.DataFrame(data=new)
    newtabs[name] = tab2
    htmls[name] = tab2.to_html(justify='left',notebook=True)
    latex_name = name+"_latex.txt"
    file = open(latex_name,"w")
    file.write(tab2.to_latex(float_format="%.3e"))
    file.close()
    
    f_html = open(name+".html","w")
    f_html.write("<!DOCTYPE html>\n")
    f_html.write('<html>\n')
    f_html.write('<body>\n')
    print(tab2.to_html(justify="left",float_format="%.2f"),file=f_html)
    f_html.write('</body>\n')
    f_html.write('</html>')
    f_html.close()

In [11]:
keys = list(newtabs.keys())
eff_mtx = {}
eff_mtx["ctau"] = [1,10,100,1000]

for i in range(len(newtabs['Mchi-22p0_dMchi-4p0_cutflow']['Cut'])):
    for key in keys:
        tab = newtabs[key]
        mass = key.split("_")[0].split("-")[1]
        cts = [1,10,100,1000]
        names = ["sig_"+mass+"_{0}_eff".format(ct) for ct in cts]
        effs_by_ct = []
        for n in names:
            effs_by_ct.append(tab[n][i])
        eff_mtx["sig_"+mass] = effs_by_ct
        eff_df = pd.DataFrame(data=eff_mtx)
        f = open("efficiency_mtx_cut{0}.html".format(i),"w")
        print(eff_df.to_html(),file=f)
        f.close()

In [10]:
len(newtabs['Mchi-22p0_dMchi-4p0_cutflow']['Cut'])

30